In [15]:
# Creating a new ABM using a network structure

In [32]:
# loading packages
import random
import numpy as np
import networkx as nx
import pandas as pd
from collections import Counter

import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import StagedActivation, BaseScheduler
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector

# CPS workflow

## World 


## Agents
Universal parameters:
problem_solving
communication

role_correct
role_distribution
role_negotiation lvl3
role_monitoring


Level 1

Level 2

Level 3
role_distributed=1

Level 4
Role_effect=1





## Teams are formed

## Problems
All team is posed with a problem with Task_complexity (1:3) and Size (1000)


Then we distribute role: 1 to len(team) (Do you fulfill your role or not?)
- each is assigned a "correct" role
- Then roles are more probable to become correct based on lvl2-4

- If Lvl 3/4 then roles are distributed correctly/evenly at first
- Self.Role=X

First we try to generate common-ground:
- Only lvl 2-4
- Based on agents Communication parameter
- Common_ground = 0 or 1

Then we simulate role_alignment:
- There some 0.8 prob that role stays correct
- Theres some 0.2 prob that role might shift to incorrect

Then Solutions are generated:
- generate some 50 to 150 solution (based on problem_solving)
- If lvl doesnt match task lvl then solution cannot be given

If some role > 1:
- Then conflict appears

If common_ground=0
- Lowest solution is chosen

If all roles are evenly distributed and common ground = 1:
    Then choose the highest solution

Adjustement of error/Meta negotiations/Self and team monitoring:
- Resolve_conflict
    - 50% prob of conflict resolvement
    - 90% if lvl 3 is present
- Reach_common_ground
    - 


In [33]:
# Simulating agents parameters

# generating pisa mean scores
def generate_pisa_score():
    score = np.random.normal(500, 95)
    return score

# generating cps levels
def generate_cps_level(x):
   # if x is below 340 then cps level is 0
   # if x is between 340 and 440 then cps level is 1
    # if x is between 440 and 540 then cps level is 2
    # if x is between 540 and 640 then cps level is 3
    # if x is above 640 then cps level is 4
    x = np.random.normal(500, 95)
    if x < 340:
        cps_level = 0
    elif x >= 340 and x < 440:
        cps_level = 1
    elif x >= 440 and x < 540:
        cps_level = 2
    elif x >= 540 and x < 640:
        cps_level = 3
    elif x >= 640:
        cps_level = 4
    return cps_level

# generating cps levels
def generate_cps_level_discrete():
   # generate a 0 with a 6% probability 
   # a 1 with a 22% probability 
   # and a 2 with a 36% probability
   # and a 3 with a 28% probability
   # and a 4 with a 8% probability
    return np.random.choice([0,1,2,3,4], p=[0.06, 0.22, 0.36, 0.28, 0.08])

def cps_weight(cps_level):
    if cps_level == 0:
        weight = random.uniform(0.5, 1)
    elif cps_level == 1:
        weight = random.uniform(1, 1.25)
    elif cps_level == 2:
        weight = random.uniform(1.25, 1.5)
    elif cps_level == 3:
        weight = random.uniform(1.75, 2)
    elif cps_level == 4:
        weight = random.uniform(2, 2.25)
    
    return weight

# Generating a function which creates a task work level which is a random number between 0 and 1 and then multiplied by x and add uncertainty around this estimate of 0.77


In [52]:
class TeamAgent(Agent):
    """ An agent with X."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.pisa_score = generate_pisa_score()
        self.cps_level = generate_cps_level(self.pisa_score)
        self.cps_weight = cps_weight(self.cps_level)
        
        self.team_members = []
        self.team_id = None
        self.team_cps = []
        
        self.current_role = ()
        self.correct_role = ()
        self.is_role_correct = 0

        self.solution = 0

    # Preparing agents into teams
    def reset_team(self):
        self.team_members = []
        self.team_id = None


    ## Joining teams
    def join_teams3(self):
        self.team_id = self.model.agent_team_dict.get(self.unique_id)

        #other_agent_ids = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id and agent_id != self.unique_id]
        #self.team_members = other_agent_ids + [self.unique_id]
        members = [agent_id for agent_id, t_id in self.model.agent_team_dict.items() if t_id == self.team_id]
        self.team_members = members

        print("agent_id:", self.unique_id, "Team_id:", self.team_id, "Team_members:", self.team_members)

    ## Creating a correct role
    def create_roles(self):

        x = list( range(1, len(self.model.teams[self.team_id])) )
        self.model.teams[self.team_id].append()
    

        self.model.teams[self.team_id]

    def get_correct_role(self):
        
        for i in self.team_members:
            self.correct_role = self.team_members.index(self.unique_id)

        print("Your correct role is:", self.correct_role)
    

    def grab_team_cps(self):
        self.team_cps = []

        for i in self.team_members:
            self.team_cps.append( self.model.schedule.agents[i].cps_level )

        print("This is your own cps:", self.cps_level)
        print("This is your weight", self.cps_weight)
        print("This is the cps of whole team:", self.team_cps)
        
    # Defining the problem loop where evaluation takes place

    def get_start_role(self):
        # Simulate a current_role for each agent from 0 to length of team_members the curren_role should have a 80% probability of being correct_role and a 20% probability of being a random role however if there is a 3 or 4 in the self.team_cps then current_role should be correct_role
        if 3 in self.team_cps or 4 in self.team_cps:
            self.current_role = self.correct_role
        else:
            self.current_role = np.random.choice([self.correct_role, random.randint(0, len(self.team_members)-1)], p=[0.8, 0.2])
        
        print("Your starting role is:", self.current_role)

    def get_current_role(self):
        # Simulate a current_role for each agent from 0 to length of team_members the curren_role should have a 80% probability of being correct_role and a 20% probability of being a random role
        alist = list(range(0, len(self.team_members)))
        other_roles = alist.remove(self.correct_role)
        self.current_role = np.random.choice([self.correct_role, other_roles], p=[0.8, 0.2])
        print("Your current role is:", self.current_role)
       
    def establish_common_ground(self):
        # common ground might be = 1 with a default prob of 50% however if there is a cps_lvl 2 in the team_members then add the weighted probability of cps_lvl

         prob = 0.5
         for i in self.team_cps:
            for i in team:
                if i > 1:
                    prob *= (1.05 ** i) # The combination of "to the power of" and 1.05 creates a good balanced probability of synergy based on the incremental charecteristics
                
    # this loop creates the problem, the roles, common ground etc.
    def problem_loop(self):

        # creating the roles
        if i == 1:
            self.get_start_role()
        else:
            self.get_current_role()

        # Creating the solution
        
        # If the task fits level
        if self.cps_level >= self.model.problem_complexity:
            self.solution = random.uniform(0.5, 1.5) * self.pisa_score
        # If the task is too complex
        elif self.cps_level < self.model.problem_complexity:
            self.solution = 0 # FOR NOW! this will remain 0 I might change this to a prob 
        
        # add the self solution to a model class variable which is indexed by the team_id
        self.model.team_solutions[self.team_id] = self.solution
        
        print("agent", self.unique_id, "team", self.team_id, "This is my solution:", self.solution)


    # evaluating and comparing solutions 
    def evaluate(self):
        # print the solutions of all agents in the team
        print("This is the team solutions:", self.model.team_solutions)

        # evaluate all solutions in the team_solutions dictionary indexed by team_id and store the highest in a model class variable called best_solution indexed by team_id
        self.model.best_solution[self.team_id] = max(self.model.team_solutions.values())

        # print the team id and the best solution in the same line
        print("This is the best solution:", self.model.best_solution[self.team_id])

        

    def test_loop(self):
        print("This is the loop")

    def test_loop2(self):
        print("This is the loop 2")

    def stage_one(self):
        self.reset_team()
        self.join_teams3()
        self.get_correct_role()
        self.grab_team_cps()
    
    def stage_two(self):
        self.test_loop()
    
    
    def stage_three(self):
        self.test_loop2()
    
    def step(self):
        self.stage_one()
        for i in range(self.model.rounds):
            self.stage_two()

    

        
       

In [72]:
from mesa.time import StagedActivation

class CustomStagedActivation(StagedActivation):
    def __init__(self, model, stage_list, num_rounds, shuffle=True, shuffle_between_stages=True):
        super().__init__(model, stage_list, shuffle, shuffle_between_stages)
        self.num_rounds = num_rounds

    def step(self):
        """Executes all the stages for all agents, including a loop alternating between stages two and three."""
        # To be able to remove and/or add agents during stepping
        # it's necessary to cast the keys view to a list.
        agent_keys = list(self._agents.keys())
        if self.shuffle:
            self.model.random.shuffle(agent_keys)
        for stage in self.stage_list:
            if stage == "stage_two":
                for _ in range(self.num_rounds):  # Loop for stage two and three (num_rounds times)
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], 'stage_two')()  # Run stage
                    for agent_key in agent_keys:
                        if agent_key in self._agents:
                            getattr(self._agents[agent_key], "stage_three")()  # Run stage
                    
            else:
                for agent_key in agent_keys:
                    if agent_key in self._agents:
                        getattr(self._agents[agent_key], stage)()  # Run stage
            # We recompute the keys because some agents might have been removed
            # in the previous loop.
            agent_keys = list(self._agents.keys())
            if self.shuffle_between_stages:
                self.model.random.shuffle(agent_keys)
            self.time += self.stage_time

        self.steps += 1


In [73]:
# Creating the model
class TeamModel(Model):
    def __init__(self, num_agents, rounds, problem_complexity_distribution):
        self.num_agents = num_agents
        self.rounds = rounds

        self.schedule = CustomStagedActivation(self, stage_list=["stage_one", "stage_two", "stage_three"], num_rounds=self.rounds)
        self.schedule2 = StagedActivation(self, stage_list=["stage_two"])
        
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []

        self.problem_complexity_distribution = problem_complexity_distribution

        self.team_solutions = {}
        self.best_solution = {}
        
        
        for i in range(self.num_agents):
            agent = TeamAgent(i, self)
            self.schedule.add(agent)

    # Creating team lists
    def create_teams(self):
        # Creating random teams of 2-5 agents based on the number of agents
        numbers = []
        x = 1
        while len(numbers) < self.num_agents:
            repetitions = random.randint(2, 5)
            numbers.extend([x] * repetitions)
            x += 1

        numbers = numbers[:self.num_agents]  # Trim the list to the desired number of agents
            
        random.shuffle(numbers) 

        self.groups = numbers

        # Creating the roles such that
        # each team has 2 to 5 roles specified by 1:n

        all_agents = list( range(0, self.num_agents) )
        self.teams = {
            "agent_id": all_agents,
            "team_id": self.groups
        }

        self.agent_team_dict = {agent_id: team_id for agent_id, team_id in zip(self.teams['agent_id'], self.teams['team_id'])}
        

    
    # creating a problem which is then used for solving
    def generate_problem(self): 
        np.random.choice([1,2,3], p=self.problem_complexity_distribution)

    def get_agent_by_id(self, agent_id):
        return self.schedule._agents[agent_id]  # Accessing agent using its unique_id
    
    # A function which checks
    def collect_teams(self):
        print(self.teams)
        print(self.agent_team_dict)

    def transfer_agents1(self):
        for i in range(self.num_agents):
            agent = TeamAgent(i, self)
            self.schedule.add(agent)

    def reset_variables(self):
        self.teams = {}
        self.agent_team_dict = {}
        self.groups = []
        self.problem_complexity = 0

    def step(self):
        self.reset_variables()
        self.create_teams()
        self.schedule.step()
        # create a loop which initiates the loop for self.rounds
        self.collect_teams()




In [74]:

model = TeamModel(num_agents=20, rounds=10, problem_complexity_distribution=[0.333, 0.333, 0.333])

for i in range(2):
    model.step()



agent_id: 16 Team_id: 1 Team_members: [7, 11, 16, 17, 18]
Your correct role is: 2
This is your own cps: 2
This is your weight 1.4587568676830813
This is the cps of whole team: [2, 1, 2, 3, 1]
agent_id: 4 Team_id: 3 Team_members: [0, 4, 12, 15, 19]
Your correct role is: 1
This is your own cps: 4
This is your weight 2.2474599216068647
This is the cps of whole team: [2, 4, 2, 1, 3]
agent_id: 10 Team_id: 2 Team_members: [5, 10]
Your correct role is: 1
This is your own cps: 3
This is your weight 1.8969869996319735
This is the cps of whole team: [1, 3]
agent_id: 19 Team_id: 3 Team_members: [0, 4, 12, 15, 19]
Your correct role is: 4
This is your own cps: 3
This is your weight 1.8231224769440972
This is the cps of whole team: [2, 4, 2, 1, 3]
agent_id: 7 Team_id: 1 Team_members: [7, 11, 16, 17, 18]
Your correct role is: 0
This is your own cps: 2
This is your weight 1.4187928356930608
This is the cps of whole team: [2, 1, 2, 3, 1]
agent_id: 1 Team_id: 4 Team_members: [1, 6, 8, 13, 14]
Your corre